<a href="https://colab.research.google.com/github/rzorzal/redes_neurais_trabalho_1/blob/master/redes_neurais_trabalho_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Trabalho de Redes Neurais 1